In [2]:
import pandas as pd
import numpy as np

# Challenge: Clean Ames

* Create a cell containing all the imports dor this notebook
* Create a cell with the `clean_housing` and `shrink_ints` functions
* Add code to load the raw data and create a `housing` variable from calling `clean_housing`
* Move those cells to the top of the notebook
* Restar the notebook and make sure that those cells work

In [4]:
path = "/home/dang/Learning/Python Statistics Essential Training/python-statistics-essential-training-4433355-main/data/ames-housing-dataset.zip"
df = pd.read_csv(path, engine="pyarrow", dtype_backend="pyarrow")

df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141,31770,Pave,<NA>,IR1,Lvl,...,0,<NA>,<NA>,<NA>,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80,11622,Pave,<NA>,Reg,Lvl,...,0,<NA>,MnPrv,<NA>,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81,14267,Pave,<NA>,IR1,Lvl,...,0,<NA>,<NA>,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93,11160,Pave,<NA>,Reg,Lvl,...,0,<NA>,<NA>,<NA>,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74,13830,Pave,<NA>,IR1,Lvl,...,0,<NA>,MnPrv,<NA>,0,3,2010,WD,Normal,189900


In [5]:
for size in [np.uint8, np.uint16, np.uint32]:
    print(np.iinfo(size))

Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for uint16
---------------------------------------------------------------
min = 0
max = 65535
---------------------------------------------------------------

Machine parameters for uint32
---------------------------------------------------------------
min = 0
max = 4294967295
---------------------------------------------------------------



In [10]:
# Create the function

def shrink_ints(df):
    mapping = {}
    for col in df.dtypes[df.dtypes == "int64[pyarrow]"].index:
        max_ = df[col].max()
        min_ = df[col].min()
        if min_ < 0:
            continue
        if max_ < 255:
            mapping[col] = "uint8[pyarrow]"
        elif max_ < 65535:
            mapping[col] = "uint16[pyarrow]"
        elif max_ < 4294967295:
            mapping[col] = "uint32[pyarrow]"
    return df.astype(mapping)

def clean_housing(df):
    return (df
            .assign(**df.select_dtypes("string").replace("", "Missing").astype("category"),
                    **{"Garage Yr Blt": df["Garage Yr Blt"].clip(upper=df["Year Built"].max())})
            .pipe(shrink_ints)    
        )

In [14]:
housing = clean_housing(df)
housing.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141,31770,Pave,<NA>,IR1,Lvl,...,0,<NA>,<NA>,<NA>,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80,11622,Pave,<NA>,Reg,Lvl,...,0,<NA>,MnPrv,<NA>,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81,14267,Pave,<NA>,IR1,Lvl,...,0,<NA>,<NA>,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93,11160,Pave,<NA>,Reg,Lvl,...,0,<NA>,<NA>,<NA>,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74,13830,Pave,<NA>,IR1,Lvl,...,0,<NA>,MnPrv,<NA>,0,3,2010,WD,Normal,189900


In [15]:
housing.dtypes

Order             uint16[pyarrow]
PID               uint32[pyarrow]
MS SubClass        uint8[pyarrow]
MS Zoning                category
Lot Frontage      uint16[pyarrow]
                       ...       
Mo Sold            uint8[pyarrow]
Yr Sold           uint16[pyarrow]
Sale Type                category
Sale Condition           category
SalePrice         uint32[pyarrow]
Length: 82, dtype: object